In [1]:
cd C:\Users\Administrator\Desktop\ML assignment\Assignment3\ml-latest-small

C:\Users\Administrator\Desktop\ML assignment\Assignment3\ml-latest-small


In [2]:
import pandas as pd 
import numpy as np

In [3]:
movies = pd.read_csv('movies.csv')
ratings = pd.read_csv('ratings.csv')
tags = pd.read_csv('tags.csv')
link = pd.read_csv('links.csv')

In [4]:
ratings.head()

,userId,movieId,rating,timestamp
0,1,1,4.0,964982703
1,1,3,4.0,964981247
2,1,6,4.0,964982224
3,1,47,5.0,964983815
4,1,50,5.0,964982931


In [5]:
ratings_count = ratings.groupby(by=['movieId'])['rating'].count().reset_index().sort_values(by = 'rating' , ascending=False)
ratings_count.head()

,movieId,rating
314,356,329
277,318,317
257,296,307
510,593,279
1938,2571,278


In [6]:
movies_count_merge = ratings_count.merge(ratings , left_on = 'movieId' , right_on='movieId' )
movies_count_merge.head()

,movieId,rating_x,userId,rating_y,timestamp
0,356,329,1,4.0,964980962
1,356,329,6,5.0,845553200
2,356,329,7,5.0,1106635915
3,356,329,8,3.0,839463527
4,356,329,10,3.5,1455301685


In [7]:
movies_rename_merge = movies_count_merge.rename(columns={'rating_x':"total_rating_count" , 'rating_y':'rating'})
movies_rename_merge.head()

,movieId,total_rating_count,userId,rating,timestamp
0,356,329,1,4.0,964980962
1,356,329,6,5.0,845553200
2,356,329,7,5.0,1106635915
3,356,329,8,3.0,839463527
4,356,329,10,3.5,1455301685


In [8]:
movie_pivot_table = movies_rename_merge.pivot_table(index = "userId" , columns = "movieId" , values = 'total_rating_count').fillna(0)
movie_pivot_table.head() 

movieId,1,2,3,4,5,6,7,8,9,10,...,193565,193567,193571,193573,193579,193581,193583,193585,193587,193609
userId,,,,,,,,,,,,,,,,,,,,,
1,215.0,0.0,52.0,0.0,0.0,102.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
5,215.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [9]:
from scipy.sparse import csr_matrix

movie_features_df_matrix = csr_matrix(movie_pivot_table.values)

from sklearn.neighbors import NearestNeighbors


model_knn = NearestNeighbors(metric = 'cosine', algorithm = 'brute')
model_knn.fit(movie_features_df_matrix)

NearestNeighbors(algorithm='brute', leaf_size=30, metric='cosine',
                 metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                 radius=1.0)

In [10]:
query_index = np.random.choice(movie_pivot_table.shape[0])
print(query_index)
distances, indices = model_knn.kneighbors(movie_pivot_table.iloc[query_index,:].values.reshape(1, -1), n_neighbors = 6)

472


In [11]:
for i in range(0, len(distances.flatten())):
    if i == 0:
        print('Recommendations for {0}:\n'.format(movie_pivot_table.index[query_index]))
    else:
        print('{0}: {1}, with distance of {2}:'.format(i, movie_pivot_table.index[indices.flatten()[i]], distances.flatten()[i]))

Recommendations for 473:

1: 250, with distance of 0.5093949697261109:
2: 449, with distance of 0.5146649268215021:
3: 258, with distance of 0.5334916474716127:
4: 65, with distance of 0.5361085038890961:
5: 396, with distance of 0.5383788800395668:


In [12]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
from surprise.model_selection import train_test_split

In [16]:
reader = Reader(rating_scale = (1, 5))
data = Dataset.load_from_df(ratings[['userId', 'movieId', 'rating']], reader)

In [18]:
[trainset, testset] = train_test_split(data, test_size = 0.15, shuffle = True)

recom = KNNWithMeans(k = 70, sim_options = {'Name' : 'Cosine', 'user_based' : True})

recom.fit(data.build_full_trainset())

test_pred = recom.test(testset)

RMSE = accuracy.rmse(test_pred)

print('RMSE = ',RMSE)

Computing the msd similarity matrix...
Done computing similarity matrix.
RMSE: 0.7217
RMSE =  0.7217078591994062


In [19]:
uid = int(input('Enter the UserId : '))
mid = int(input('Enter the MovieId : '))
pre = recom.predict(uid, mid)
print('The rating value for MovieId ',mid,' by UserId ',uid, ' is ',pre[3])

Enter the UserId : 6
Enter the MovieId : 396
The rating value for MovieId  396  by UserId  6  is  3.501556983616962


In [20]:
user_id = int(input('Enter user ID : '))
movie_id = int(input('Enter movie ID : '))
print('The rating for User ID : {0} and Movie ID {1} is {2}'.format(user_id, movie_id, round((recom.predict(user_id,movie_id)[3]),2)))


Enter user ID : 7
Enter movie ID : 271
The rating for User ID : 7 and Movie ID 271 is 3.0


In [22]:
recom.predict(user_id, movie_id)[3]


2.9979522806461802

In [31]:
# Keep a threshold of 2.5 and then display recommendations

def recommend(pre):
    if pre[3] >= 2.5 :
        return True
    else:
        return False

In [37]:
uid = int(input('Enter the UserId : '))
d = dict()
for i in movie_pivot_table.columns:
    pre = recom.predict(uid,i)
    if recommend(pre) == True :
        d[i] = pre[3]
        
final_dict = sorted(d.items(), key=lambda x: x[1], reverse=True)

Enter the UserId : 600


In [41]:
print('Top 10 movies recommend for UserId ',uid,' :')
for i in range(10):
    print(i+1,' --> ',(movies[movies['movieId'] == final_dict[i][0]]['title'].values)[0])

Top 10 movies recommend for UserId  600  :


KeyError: 'movieId'